In [1]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
import kagglehub
path = kagglehub.dataset_download("hijest/englishrussian-dictionary-for-machine-translate")
print("Path to dataset files:", path)

Using Colab cache for faster access to the 'englishrussian-dictionary-for-machine-translate' dataset.
Path to dataset files: /kaggle/input/englishrussian-dictionary-for-machine-translate


In [4]:
import string
import re
from tensorflow import keras
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.utils import plot_model
import pandas as pd

# Путь к вашему файлу
text = "/kaggle/input/englishrussian-dictionary-for-machine-translate/rus.txt"

# Чтение файла и обработка строк
with open(text, encoding='utf-8') as file:
    lines = file.read().split("\n")

pairs = []

for line in lines:
    parts = line.split("\t")
    if len(parts) >= 2:
        english, russian = parts[0], parts[1]
        # Цель — английский русский
        english = "[start] " + english + " [end]"
        pairs.append((russian, english))

In [5]:
df = pd.DataFrame(pairs)
df.head(20)

,0,1
0,Марш!,[start] Go. [end]
1,Иди.,[start] Go. [end]
2,Идите.,[start] Go. [end]
3,Здравствуйте.,[start] Hi. [end]
4,Привет!,[start] Hi. [end]
5,Хай.,[start] Hi. [end]
6,Здрасте.,[start] Hi. [end]
7,Здоро́во!,[start] Hi. [end]
8,Беги!,[start] Run! [end]
9,Бегите!,[start] Run! [end]
